In [1]:
import pandas as pd 

books = pd.read_csv("books_with_categories.csv")

In [2]:
from transformers import pipeline

classifier = pipeline(
                        "text-classification", 
                        model="j-hartmann/emotion-english-distilroberta-base",
                        top_k = None)

classifier("I love this!")


/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528691716492176},
  {'label': 'neutral', 'score': 0.005764594301581383},
  {'label': 'anger', 'score': 0.004419787786900997},
  {'label': 'sadness', 'score': 0.002092392183840275},
  {'label': 'disgust', 'score': 0.0016119909705594182},
  {'label': 'fear', 'score': 0.0004138525982853025}]]

### Apply Sentiment Analysis on every description

In [ ]:
import numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

# This function only takes one description and then gives sentence a score in the emotions.
def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels} # Dictionay with emotion scores for each sentence
    
    for prediction in predictions:                               # Loop over every sentence in the description
        sorted_predictions = sorted(prediction, key=lambda x: x["label"]) 
        
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"]) # For every label append the score
    
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()} # Take maximum score for each emotion

In [6]:
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}


for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".") # Splitting the description into sentences.
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [16:58<00:00,  5.10it/s]


In [7]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

In [8]:
books = pd.merge(books, emotions_df, on="isbn13")

In [9]:
books.to_csv("books_with_emotions.csv", index=False)